In [6]:
#import modules
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
import os, shutil, stat
import glob as gb
import cv2
import tensorflow as tf
import keras
from tqdm import tqdm



ImportError: cannot import name 'LazyLoader' from 'keras.utils.generic_utils' (C:\Users\Dell\.conda\envs\my-env\lib\site-packages\keras\utils\generic_utils.py)

In [5]:
!pip install keras.api

ERROR: Could not find a version that satisfies the requirement keras.api
ERROR: No matching distribution found for keras.api


In [3]:
!pip install opencv-python

In [ ]:
!pip install tqdm

# Now we get the dataset (my first dataset too)

In [ ]:
path = 'C:\\Users\\Dell\\Desktop\\huawi\\' #to get the path of folders 
Normalimages = os.listdir(path + "\\normal")
AbnormalImages = os.listdir(path + "\\abnormal")


# Now we get to the 2 folders 

In [ ]:
for folder in  os.listdir(path) :    
    files = os.listdir(os.path.join(path,folder))
    print(f'For data , found {len(files)} in folder {folder}')  #to get the numbers of images in each folder

# We must make our dictionary for labeling the data

In [ ]:
code = {'normal':0 ,'abnormal':1}   #Dictionary  for labeling data

def getcode(n) : 
   
            return code[n]     #get the code of data from dictionary
        
def getLabel(n):
    for x,c in new_labels.items():
        if n==c:
            return x

        
        


# Now we read the images from 2 folders

In [ ]:
size = []
images=[] # to store images
labels=[] # to store label for each image
for folder in  os.listdir(path) : 
    files = os.listdir(os.path.join(path,folder))
    
    
    for file in tqdm(files): 
        image=plt.imread(os.path.join(path,folder+"/"+file)) 
        images.append(image) # add image into list 
        labels.append(folder) # to add label into list 
#         size.append(image.shape)



In [ ]:
# len(images ), len(labels),labels

Now we plot to know the number of images in each flder and mak a simple comparison

In [ ]:
#show 
sns.countplot(labels)

# Now we show a sample of images.
# Note 1 for abnormal and 0 for normal

In [ ]:

plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(images),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(images[i])    
    plt.axis('off')
    plt.title(getcode(labels[i]))

# Let's make the list of images into arrays

In [ ]:
            
images=np.array(images)
labels=np.array(labels)

plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(images),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(images[i])    
    plt.axis('off')
    plt.title(getcode(labels[i]))    

# The shape of images isn't the same so i resize them to make them having same size 

In [ ]:
#resize images into 224 ,224, 3
image_size=224

new_images=[]
new_labels=[]

for i in range(len(images)):
    
    if images[i].shape[-1]!=image_size : # to exclude it image that has 1 channel (grey scale)
        if images[i].shape[-1] ==3: # to select the image that has 3 channel  (some of them have 4)
            new_images.append(cv2.resize(images[i] , (image_size,image_size)) )
            new_labels.append(labels[i])
            
            
            
            
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(new_images),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(new_images[i])    
    plt.axis('off')
    plt.title(getcode(new_labels[i]))            

# convert labels to one hot encoder

In [ ]:
# convert labels to one hot encoder
new_labels=[code[item] for item in new_labels]

# In this section i make the resized images into array

In [ ]:
# convert data to array , and rescale images 
new_labels=np.array(new_labels)

new_images=np.array(new_images)/255.0

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(new_images),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(new_images[i])    
    plt.axis('off')
    plt.title(new_labels[i])   

# Now we finish the data preparation and scaling.
# Let's split it into train, validate and test 

In [ ]:

from sklearn.model_selection import train_test_split
# split entire data to x_train , x_test , y_train , y_test 
x_train , x_test , y_train , y_test =train_test_split(new_images , new_labels , test_size=0.2 , shuffle=True , stratify=new_labels)

# split entire train to x_train , x_val , y_train , y_val

x_train , x_val , y_train , y_val=train_test_split(x_train , y_train , test_size=0.05 , shuffle=True , stratify=y_train)

# 

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(y_train),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(x_train[i])    
    plt.axis('off')
    plt.title(y_train[i])   

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

generator=ImageDataGenerator(
       
      rotation_range=90,
      width_shift_range=0.1,
      height_shift_range=0.1,
    vertical_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
)

batch_size=50                                            
train_generator=generator.flow(x_train,y_train ,batch_size=batch_size)
val_generator=generator.flow(x_val,y_val)

#get data for generator
train_data=train_generator.__getitem__(0)[0]
train_labels=train_generator.__getitem__(0)[1]


In [ ]:
            
# plt.figure(figsize=(20,20))
# for n , i in enumerate(list(np.random.randint(0,32,36))) : 
#     plt.subplot(6,6,n+1)
#     plt.imshow(train_data[i])    
#     plt.axis('off')
#     plt.title(train_labels[i])    

# build models 

In [ ]:
# model = tf.keras.models.Sequential([
# # YOUR CODE HERE
# tf.keras.layers.Conv2D(1024,kernel_size=(3,3),activation='relu',input_shape=(image_size,image_size,3)),
# tf.keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu'),
# tf.keras.layers.AvgPool2D(4,4),
# tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu'),    
# tf.keras.layers.Conv2D(80,kernel_size=(3,3),activation='relu'),    
# tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
# tf.keras.layers.MaxPool2D(4,4),
# tf.keras.layers.Flatten(),
# #Full Connected Layers
# tf.keras.layers.Dense(512, activation='relu'),
# #Add dropout to avoid Overfit
# tf.keras.layers.Dropout(0.25),
# tf.keras.layers.Dense(256, activation='relu'),
# tf.keras.layers.Dense(128, activation='relu'),
# #Add dropout to avoid Overfit
# tf.keras.layers.Dropout(0.4),
# tf.keras.layers.Dense(64, activation='relu'),
# tf.keras.layers.Dense(1 , activation='sigmoid'),
       
# ])

# print(model.summary())
# model.compile(optimizer='adam', loss="binary_crossentropy",metrics=['accuracy'])

# epochs = 64
# history = model.fit(x_train , y_train ,batch_size=32,epochs=epochs , validation_data=(x_val , y_val))


In [ ]:
# !pip install efficientnet

In [ ]:
# !pip install segmentation_models

In [ ]:
# pip install keras==2.2.4

In [ ]:
!pip install q tensorflow==2.1
!pip install q keras==2.3.1

In [ ]:

# from keras.applications import VGG16

# pretrain_model = VGG16(input_shape = (image_size, image_size, 3), weights = 'imagenet', include_top = False)

# for layer in pretrain_model.layers:
#     layer.trainable = False
    
# pretrain_model.summary()    

# from keras.applications import ResNet50

# # resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# pretrain_model=ResNet50(include_top=False,input_shape=(image_size,image_size,3),pooling='max',classes=2, weights = 'imagenet')
# for layer in pretrain_model.layers:
#     layer.trainable = False
# pretrain_model.summary()    


import efficientnet.keras as efn 
pretrain_model = efn.EfficientNetB3(
    weights='imagenet',
    input_shape=(image_size,image_size,3),
    include_top=False,
    pooling='avg'
)

pretrain_model.summary()    


In [ ]:
from tensorflow.keras import Model

x = tf.keras.layers.Flatten()(pretrain_model.output)

#Full Connected Layers
x = tf.keras.layers.Dense(512, activation='relu')(x)
#Add dropout to avoid Overfit
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
#Add dropout to avoid Overfit
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)


x=tf.keras.layers.Dense(1 , activation='sigmoid')(x)
       

model = Model( pretrain_model.input, x) 

print(model.summary())
model.compile(optimizer='adam', loss="binary_crossentropy",metrics=['accuracy'])

epochs = 20
history = model.fit(x_train , y_train ,batch_size=20,epochs=epochs , validation_data=(x_val , y_val))

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
model.save('./saveModel.h5')

In [ ]:
from tensorflow.keras import Model
from keras.models import load_model
loadedModel=load_model("./saveModel.h5")
loadedModel.compile(optimizer='adam', loss="binary_crossentropy",metrics=['accuracy'])
model.evaluate(x_test, y_test)

In [ ]:
pred=model.predict(x_test)

In [ ]:
new_pred=[]
for p in pred:
    if (p >= 0.5):
        new_pred.append(1)
    else:
        new_pred.append(0)
        
print(new_pred)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['normal', 'abnormal']
print(classification_report(new_pred,y_test, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(new_pred,y_test)
print(cm)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['val'], loc='upper left')
plt.show()




In [ ]:
def gellabel(arr):
    new_arr=[]
    for p in arr:
        if (p == 0):
            new_arr.append('normal')
        else:
            new_arr.append('abnormal')
        
    return new_arr

In [ ]:
new_y_test= gellabel(y_test)
new_new_pred=gellabel(new_pred)
plt.figure(figsize=(30,30))
for n , i in enumerate(list(np.random.randint(0,32,36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(x_test[i])    
    plt.axis('off')
    plt.title(f"   Real: {(new_y_test[i])   } Vs  Predict: {(new_new_pred[i])}")
